<a href="https://colab.research.google.com/github/as9786/ML-DLPratice/blob/main/Pytorch/Boston_housing_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
(train_data, train_targets), (test_data, test_targets) = keras.datasets.boston_housing.load_data()

57026/57026 [==============================] - 0s 0us/step


# Module

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import numpy as np

In [ ]:
print(len(train_targets))
print(len(test_targets))

404
102


In [ ]:
data = list(train_data) + list(test_data)
target = list(train_targets) + list(test_targets)
len(data), len(target)

(506, 506)

In [ ]:
data = np.array(data)
target = np.array(target)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_valid,y_train,y_valid = train_test_split(data,target,test_size=0.3)
X_valid,X_test,y_valid,y_test = train_test_split(X_valid,y_valid,test_size=0.35)

print(len(X_train))
print(len(X_valid))
print(len(X_test))

354
98
54


In [ ]:
type(X_train)
type(y_train)

numpy.ndarray

In [ ]:
X_train.shape
y_train.shape

(354,)

# Torch dataset

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
class BostonDataset(Dataset):
    scaler = StandardScaler()
    def __init__(self,X,y,phase):
        self.X = X
        self.y = y
        if phase == 'train':
            self.X = BostonDataset.scaler.fit_transform(self.X)
        else:
            self.X = BostonDataset.scaler.transform(self.X)

    def __len__(self): 
        assert len(self.X) == len(self.y)
        return len(self.X)

    def __getitem__(self,idx): 
        data = torch.tensor(self.X[idx],dtype=torch.float)
        price = torch.tensor(self.y[idx],dtype=torch.float)
        # data = self.X[idx]
        # price = self.y[idx]
        return data, price

In [ ]:
train_dataset = BostonDataset(X_train,y_train,'train') 
valid_dataset = BostonDataset(X_valid,y_valid,'valid')
test_dataset = BostonDataset(X_test,y_test,'test')

In [ ]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
valid_loader = DataLoader(valid_dataset,batch_size=16) 
test_loader = DataLoader(test_dataset,batch_size=1)

In [ ]:
for x,y in train_loader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([32, 13])
torch.Size([32])


# Model

In [ ]:
class MLPRegressor(nn.Module):

    def __init__(self):
        super().__init__()
        self.weight_init = nn.init.xavier_uniform_
        self.linear1 = nn.Linear(13,64)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(64,64)
        self.linear3 = nn.Linear(64,1)

        # 가중치 초기화
        self.weight_init(self.linear1.weight)
        self.weight_init(self.linear2.weight)
        self.weight_init(self.linear3.weight)
        

    def forward(self,x):
        h1 = self.linear1(x)
        h1 = self.relu(h1)
        h2 = self.linear2(h1)
        h2 = self.relu(h2)
        output = self.linear3(h2)
        return output

In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
device = 'cuda'

In [ ]:
model = MLPRegressor()

In [ ]:
model.to(device)

MLPRegressor(
  (linear1): Linear(in_features=13, out_features=64, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=64, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                   Param #
MLPRegressor                             --
├─Linear: 1-1                            896
├─ReLU: 1-2                              --
├─Linear: 1-3                            4,160
├─Linear: 1-4                            65
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0

In [ ]:
loss_fn = nn.MSELoss(reduction='mean')
#loss_fn = nn.functional.mse_loss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def train(dataloader,model,loss_fn,optimizer,epoch):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader): 
        X,y = X.to(device), y.to(device)
        # 예측과 손실 계산
        pred = model(X)
        loss = loss_fn(pred,y)

        #역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss : {loss:>7f}, [{current:>5d}/{size:>5d}]')
    return loss.item()

In [ ]:
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0,0

    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device), y.to(device) 
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            
        test_loss /= num_batches
        print(f'Test loss : {test_loss:>8f}')

    return test_loss

In [ ]:
# 학습 진행하기
epochs = 100
losses = []
val_losses = []
for t in range(epochs):
    print(f'Epoch {t+1}\n----------------')
    epoch_loss = train(train_loader,model,loss_fn,optimizer)
    losses.append(epoch_loss)
    epoch_test_loss = test(valid_loader,model,loss_fn) 
    val_losses.append(epoch_test_loss)
print('Done!')

Epoch 1
----------------
loss : 77.105583, [    0/  354]
loss : 90.300621, [  320/  354]
Test loss : 82.365346
Epoch 2
----------------
loss : 58.153095, [    0/  354]
loss : 51.238708, [  320/  354]
Test loss : 82.374218
Epoch 3
----------------
loss : 41.077034, [    0/  354]
loss : 61.544395, [  320/  354]
Test loss : 82.506145
Epoch 4
----------------
loss : 70.159050, [    0/  354]
loss : 82.990807, [  320/  354]
Test loss : 83.152569
Epoch 5
----------------
loss : 98.800766, [    0/  354]
loss : 59.225609, [  320/  354]
Test loss : 82.020861
Epoch 6
----------------
loss : 68.982254, [    0/  354]


<ipython-input-149-f1e096a247c1>:7: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = loss_fn(pred,y)
<ipython-input-149-f1e096a247c1>:7: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = loss_fn(pred,y)
<ipython-input-150-72c2c018714e>:10: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  test_loss += loss_fn(pred,y).item()
<ipython-input-150-72c2c018714e>:10: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely 

loss : 96.145859, [  320/  354]
Test loss : 81.919078
Epoch 7
----------------
loss : 92.891098, [    0/  354]
loss : 55.092369, [  320/  354]
Test loss : 81.981549
Epoch 8
----------------
loss : 115.138794, [    0/  354]
loss : 120.902184, [  320/  354]
Test loss : 82.297915
Epoch 9
----------------
loss : 119.197693, [    0/  354]
loss : 50.405205, [  320/  354]
Test loss : 82.134651
Epoch 10
----------------
loss : 57.639309, [    0/  354]
loss : 87.382996, [  320/  354]
Test loss : 82.047802
Epoch 11
----------------
loss : 63.174866, [    0/  354]
loss : 81.513504, [  320/  354]
Test loss : 81.735810
Epoch 12
----------------
loss : 79.941368, [    0/  354]
loss : 96.251282, [  320/  354]
Test loss : 81.660670
Epoch 13
----------------
loss : 76.610085, [    0/  354]
loss : 103.047142, [  320/  354]
Test loss : 81.769600
Epoch 14
----------------
loss : 108.713554, [    0/  354]
loss : 45.312294, [  320/  354]
Test loss : 81.756896
Epoch 15
----------------
loss : 71.503052, [   

In [ ]:
test(test_loader,model,loss_fn,1)

Test loss : 96.113225


<ipython-input-150-72c2c018714e>:10: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  test_loss += loss_fn(pred,y).item()


96.11322501715686

In [ ]:
def value_predict(model,dataloader,i):
    X,y = dataloader.dataset.__getitem__(i)
    X,y = X.to(device), y.to(device)
    pred = model(X)
    return pred.item(),y.item()


In [ ]:
value_predict(model,test_loader,2)

(22.161277770996094, 34.599998474121094)

In [ ]:
len(losses),len(val_losses)

(100, 100)